In [31]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [32]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()

        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self , x):
        return self.disc(x)

class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()

        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(), # normalize inputs to [-1, 1] so make outputs [-1. 1] as well.
        )  

    def forward(self, x):
        return self.gen(x)              

In [33]:
lr = 3e-4
epochs=100
img_dim = 28*28*1
z_dim = 64
batch_size = 32

device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
dataset = datasets.MNIST(root='./datasets/', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

opt_disc = torch.optim.Adam(disc.parameters(), lr=lr)
opt_gen = torch.optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()

writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0

In [34]:
for epoch in tqdm(range(epochs)):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake)/2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [0/100] Batch 0/1875                       Loss D: 0.6875, loss G: 0.7646


  1%|          | 1/100 [00:11<19:17, 11.69s/it]

Epoch [1/100] Batch 0/1875                       Loss D: 0.4698, loss G: 1.0583


  2%|▏         | 2/100 [00:23<19:07, 11.71s/it]

...

 98%|█████████▊| 98/100 [19:46<00:24, 12.38s/it]

Epoch [98/100] Batch 0/1875                       Loss D: 0.5377, loss G: 0.9826


 99%|█████████▉| 99/100 [19:59<00:12, 12.44s/it]

Epoch [99/100] Batch 0/1875                       Loss D: 0.6540, loss G: 0.9556


100%|██████████| 100/100 [20:11<00:00, 12.12s/it]
